In [1]:
# write all code in one cell 

#========================Load data=========================
import numpy as np
import pandas as pd

train_data_source = 'amazon_train.csv'
test_data_source = 'amazon_valid.csv'

train_df = pd.read_csv(train_data_source)
test_df = pd.read_csv(test_data_source)

In [2]:
# to change use .astype() 
test_df['text'] = test_df.text.astype(str)
train_df['text'] = train_df.text.astype(str)

In [16]:
#train_df.head()
# test_df.dtypes

,text,label
0,I had big expectations because I love English ...,1
1,I highly recommend this series. It is a must f...,2
2,I discovered this series quite by accident. Ha...,2
3,This is the best of the best comedy Stand-up. ...,2
4,"Funny, interesting, a great way to pass time. ...",2


In [4]:
# convert string to lower case 
train_texts = train_df['text'].values 
train_texts = [s.lower() for s in train_texts] 

test_texts = test_df['text'].values 
test_texts = [s.lower() for s in test_texts] 

In [5]:
#=======================Convert string to index================
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)

# Convert string to index 
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')

In [7]:
#=======================Get classes================
train_classes = train_df['label'].values
train_class_list = [x-1 for x in train_classes]

test_classes = test_df['label'].values
test_class_list = [x-1 for x in test_classes]

from keras.utils import to_categorical
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)

In [8]:
print(tk.word_index)

{'1': 41, '$': 56, 'a': 5, '"': 32, '&': 35, '[': 59, 'j': 28, 's': 8, ';': 33, '`': 55, 'e': 3, '?': 45, '_': 54, 'z': 38, '=': 62, ',': 25, 'o': 7, '/': 48, '{': 67, 'w': 17, '-': 29, ')': 34, 'm': 16, 'g': 18, ']': 58, 'x': 30, ':': 46, '4': 40, 'u': 15, '>': 61, '\\': 66, '2': 42, "'": 27, 'i': 6, '.': 22, '5': 50, 'q': 43, '9': 51, '(': 36, '+': 57, 't': 4, 'v': 24, 'y': 20, 'b': 23, 'c': 14, ' ': 2, 'r': 10, '8': 47, '^': 68, 'd': 13, 'f': 19, '!': 31, '3': 37, '#': 39, '%': 60, 'UNK': 1, 'n': 9, 'h': 11, 'k': 26, '@': 64, '}': 65, '6': 53, '~': 63, 'p': 21, 'l': 12, '*': 52, '7': 49, '0': 44}


In [9]:
vocab_size=len(tk.word_index)
vocab_size

68

In [10]:
embedding_weights=[]

In [11]:
embedding_weights.append(np.zeros(vocab_size))

In [12]:
for char, i in tk.word_index.items(): # from index 1 to 70
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [13]:
print(embedding_weights.shape) # first row all 0 for PAD, 69 char, last row for UNK
embedding_weights

(69, 68)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [18]:
# parameter 
input_size = 1014
# vocab_size = 86
embedding_size = 68
conv_layers = [[256, 7, 3], 
               [256, 7, 3], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 2
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [19]:
# Embedding layer Initialization
embedding_layer = Embedding(vocab_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

In [20]:
# Model 

# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding 
x = embedding_layer(inputs)
# Conv 
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x) 
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x) # Final shape=(None, 34, 256)
x = Flatten()(x) # (None, 8704)
# Fully connected layers 
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x) # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) # Adam, categorical_crossentropy
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1014)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1014, 68)          4692      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1008, 256)         122112    
_________________________________________________________________
activation_1 (Activation)    (None, 1008, 256)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 336, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 330, 256)          459008    
_________________________________________________________________
activation_2 (Activation)    (None, 330, 256)          0         
__________

In [21]:
# 1000 training samples and 100 testing samples
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices][:]
y_train = train_classes[indices][:]

x_test = test_data[:]
y_test = test_classes[:]

In [ ]:
# Training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=6,
          verbose=2)

Train on 16470 samples, validate on 3294 samples
Epoch 1/6
 - 414s - loss: 0.3539 - acc: 0.8854 - val_loss: 0.3231 - val_acc: 0.8922
Epoch 2/6
 - 411s - loss: 0.3204 - acc: 0.8900 - val_loss: 0.2890 - val_acc: 0.8922
Epoch 3/6
 - 399s - loss: 0.2821 - acc: 0.8905 - val_loss: 0.2937 - val_acc: 0.9001
Epoch 4/6
 - 399s - loss: 0.2536 - acc: 0.8975 - val_loss: 0.2306 - val_acc: 0.9056
Epoch 5/6
 - 404s - loss: 0.1988 - acc: 0.9201 - val_loss: 0.2297 - val_acc: 0.9104
Epoch 6/6


In [20]:
model.save("my_model_amazon.h5")